In [145]:
!pip install catboost
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install matplotlib
!pip install seaborn
!pip install ydata-profiling
!pip install catboost category-encoders
!pip install shap
!pip install xgboost
!pip install lightgbm

    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


...

    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------------------------------

    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^


In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, classification_report
from ydata_profiling import ProfileReport
from category_encoders import CatBoostEncoder
import shap
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

In [158]:
# Чтение CSV-файла в исходной кодировке, например 'cp1251'
df = pd.read_csv('trnsctns_ops_trn.csv', encoding='cp1251')

# Сохранение в новой кодировке, например 'utf-8'
df.to_csv('trnsctns_ops_trn.csv', encoding='utf-8', index=False)


In [161]:
# Чтение CSV-файла в исходной кодировке, например 'cp1251'
df = pd.read_csv('cntrbtrs_clnts_ops_trn.csv', encoding='cp1251',sep=";")

# Сохранение в новой кодировке, например 'utf-8'
df.to_csv('cntrbtrs_clnts_ops_trn.csv', encoding='utf-8', index=False)

C:\Users\Neznayka08\AppData\Local\Temp\ipykernel_14308\2292784071.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cntrbtrs_clnts_ops_trn.csv', encoding='cp1251',sep=";")


In [148]:
df = pd.read_csv('cntrbtrs_clnts_ops_trn.csv',encoding="utf-8")
#df = df.drop(columns=['Unnamed: 0'], errors='ignore')
#df = df.drop_duplicates()
#df = df.dropna()
df= df.drop(columns=['clnt_id','accnt_id'], errors='ignore')

C:\Users\Neznayka08\AppData\Local\Temp\ipykernel_20736\1250668766.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cntrbtrs_clnts_ops_trn.csv',encoding="utf-8")


,slctn_nmbr,gndr,brth_yr,prsnt_age,accnt_bgn_date,cprtn_prd_d,erly_pnsn_flg,accnt_status,pnsn_age,prvs_npf,...,dstrct,city,sttlmnt,pstl_code,okato,phn,email,lk,assgn_npo,assgn_ops
0,0,ж,1960,64,2005-12-21,96,0,Накопительный период,55,СФР,...,NaN,ОМСК Г,NaN,644001,5.200000e+10,нет,нет,нет,нет,нет
1,0,м,1954,70,2009-06-29,283,0,Накопительный период,60,СФР,...,NaN,БЕЛОГОРСК Г,NaN,676852,1.000000e+10,нет,нет,нет,нет,нет
2,3,м,1955,69,2006-12-25,88,0,Накопительный период,60,СФР,...,NaN,NaN,NaN,109451,4.500000e+10,нет,нет,нет,нет,нет
3,2,ж,1962,62,2013-09-03,1301,0,Накопительный период,55,NaN,...,АЛЬМЕТЬЕВСКИЙ Р-Н,АЛЬМЕТЬЕВСК Г,NaN,423464,9.200000e+10,нет,нет,нет,нет,нет
4,2,м,1955,69,2010-12-06,106,0,Накопительный период,60,СФР,...,ВОТКИНСКИЙ Р-Н,NaN,МОЛЧАНЫ Д,427415,9.400000e+10,нет,нет,нет,нет,нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532820,0,ж,1965,59,2013-12-23,490,0,Накопительный период,56,СФР,...,NaN,НИЖНИЙ НОВГОРОД Г,NaN,603070,2.200000e+10,нет,нет,нет,нет,нет
532821,3,м,1956,68,2008-05-28,307,0,Накопительный период,60,СФР,...,NaN,ПЕТРОЗАВОДСК Г,NaN,185030,8.600000e+10,нет,нет,нет,нет,нет
532822,0,м,1959,65,2010-12-15,97,0,Накопительный период,60,СФР,...,ЧИШМИНСКИЙ Р-Н,NaN,АЛКИНО-2 С,452155,8.000000e+10,нет,нет,нет,нет,нет
532823,0,ж,1959,65,2010-11-03,139,0,Накопительный период,55,"НПФ ""НПФ ОПК""",...,NaN,МИЧУРИНСК Г,NaN,393761,6.800000e+10,нет,нет,нет,нет,нет


In [149]:
# Проверяем наличие целевой переменной
target = 'erly_pnsn_flg'
if target not in df.columns:
    raise KeyError(f"Целевая переменная '{target}' не найдена в DataFrame.")


In [150]:
# Определяем категориальные и бинарные столбцы
categorical_columns = ['rgn', 'dstrct', 'city', 'sttlmnt', 'addrss_type', 'prvs_npf', 'brth_plc', 'okato']
binary_columns = ["gndr", "lk"]
additional_columns = ['accnt_bgn_date', 'pstl_code', 'cprtn_prd_d', 'prsnt_age', 'pnsn_age']


In [151]:
# Этап 3: Предварительная обработка данных
df.replace({'******': np.nan}, inplace=True)  # Заменяем '******' на NaN
df.replace({'': np.nan}, inplace=True)  # Заменяем пустые строки на NaN

# Преобразование бинарных данных в числовой формат (1 и 0)
df['gndr'] = df['gndr'].map({'м': 1, 'ж': 0}).fillna(0)  # Заполняем NaN 0
df['lk'] = df['lk'].map({'да': 1, 'нет': 0}).fillna(0)  # Заполняем NaN 0

# Заполнение NaN в категориальных столбцах значением 'missing' и преобразуем их в строки
for column in categorical_columns:
    df[column] = df[column].fillna("missing").astype(str)

# Обработка даты `accnt_bgn_date`: преобразуем в год
df['accnt_bgn_date'] = pd.to_datetime(df['accnt_bgn_date'], errors='coerce')
df['accnt_bgn_year'] = df['accnt_bgn_date'].dt.year.fillna(0).astype(int)

# Обрезка `okato` до первых двух символов и преобразование в строку
df['okato'] = df['okato'].astype(str).str[:2]

# Этап 4: Объединение колонок
df['location'] = df[['rgn', 'dstrct', 'city', 'sttlmnt']].agg('-'.join, axis=1)  # Объединяем в одну колонку


In [152]:
# Кодируем категориальные признаки с помощью CatBoostEncoder
encoder = CatBoostEncoder(cols=['location', 'addrss_type', 'prvs_npf', 'brth_plc', 'okato'])
df_encoded = encoder.fit_transform(df[['location', 'addrss_type', 'prvs_npf', 'brth_plc', 'okato']], df[target])  # Кодируем только категориальные переменные


Проверка на NaN после кодирования категориальных данных:
location       0
addrss_type    0
prvs_npf       0
brth_plc       0
okato          0
dtype: int64


In [153]:
# Объединяем закодированные данные с бинарными и числовыми переменными
final_columns = binary_columns + additional_columns + ['accnt_bgn_year']
df_final = pd.concat([df_encoded, df[final_columns]], axis=1)

# Убедитесь, что целевая переменная всё ещё присутствует
if target in df.columns:
    df_final[target] = df[target]

df_final = df_final.apply(pd.to_numeric, errors='coerce').fillna(0)

# Проверяем, что все значения числовые, и заменяем любые NaN на 0
df_final = df_final.apply(pd.to_numeric, errors='coerce').fillna(0)
df_final.to_csv('processed_data.csv', encoding='utf-8', index=False)

In [155]:
# Этап 9: Генерация отчета с помощью ydata-profiling
profile = ProfileReport(df_final, title='Pandas Profiling Report', explorative=True)
profile.to_file("output_report.html")  # Сохраните отчет в HTML


c:\python\lib\site-packages\ydata_profiling\profile_report.py:357: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]